# Multiband Models

In this tutorial you will learn how to set up multiband model fits. These are formatted similarly to Group_Model objects, the main difference being that every model has a unique target to fit. That said, one "model" for a multiband model object could itself be a full Group_Model object so in truth there are no limitations to what can be fit across multiple bands! 

It is, of course, more work to set up a fit across multiple wavelength bands. However, the tradeoff can be well worth it in some cases. Perhaps there is space-based data with high resolution, but groundbased data has better S/N. Or perhaps each band individually does not have enough signal for a confident fit, but all three together just might. Perhaps colour information is of paramount importance for a science goal, one would hope that both bands could be treated on equal footing but in a consistent way when extracting profile information. There are a number of reasons why one might wish to try and fit a multiband picture of a galaxy simultaneously. 

When fitting multiple bands one often resorts to forced photometry, somtimes also blurring each image to the same approximate PSF. With AutoProf this is entirely unecessary as one can fit each image in its native PSF simultaneously. The final fits are more meaningful and can encorporate all of the available structure information.

In [ ]:
import autoprof as ap
import numpy as np
import torch
from astropy.io import fits
import matplotlib.pyplot as plt
from scipy.stats import iqr

In [ ]:
# First we need some data to work with, let's use LEDA 41136 as our example galaxy

# Our first image is from the DESI Legacy-Survey r-band. This image has a pixelscale of 0.262 arcsec/pixel and is 500 pixels across
target_r = ap.image.Target_Image(
    data = np.array(fits.open("https://www.legacysurvey.org/viewer/fits-cutout?ra=187.3119&dec=12.9783&size=500&layer=ls-dr9&pixscale=0.262&bands=r")[0].data, dtype = np.float64),
    pixelscale = 0.262,
    zeropoint = 22.5,
    variance = np.ones((500,500))*0.008**2, # note that the variance is important to ensure all images are compared with proper statistical weight. Here we just use the IQR^2 of the pixel values as the variance, for science data one would use a more accurate variance value
    psf = ap.utils.initialize.gaussian_psf(1.12/2.355, 51, 0.262) # we construct a basic gaussian psf for each image by giving the simga (arcsec), image width (pixels), and pixelscale (arcsec/pixel)
)

# The second image is a unWISE W1 band image. This image has a pixelscale of 2.75 arcsec/pixel and is 52 pixels across
target_W1 = ap.image.Target_Image(
    data = np.array(fits.open("https://www.legacysurvey.org/viewer/fits-cutout?ra=187.3119&dec=12.9783&size=52&layer=unwise-neo7&pixscale=2.75&bands=1")[0].data, dtype = np.float64),
    pixelscale = 2.75,
    zeropoint = 25.199,
    variance = np.ones((52,52))*4.9**2,
    psf = ap.utils.initialize.gaussian_psf(6.1/2.355, 21, 2.75),
    origin = (np.array([500,500]))*0.262/2 - (np.array([52,52]))*2.75/2, # here we ensure that the images line up by slightly adjusting the origin
)

# The third image is a GALEX NUV band image. This image has a pixelscale of 1.5 arcsec/pixel and is 90 pixels across
target_NUV = ap.image.Target_Image(
    data = np.array(fits.open("https://www.legacysurvey.org/viewer/fits-cutout?ra=187.3119&dec=12.9783&size=90&layer=galex&pixscale=1.5&bands=n")[0].data, dtype = np.float64),
    pixelscale = 1.5,
    zeropoint = 20.08,
    variance = np.ones((90,90))*0.0007**2,
    psf = ap.utils.initialize.gaussian_psf(5.4/2.355, 21, 1.5),
    origin = (np.array([500,500]))*0.262/2 - (np.array([90,90]))*1.5/2,
)

fig1, ax1 = plt.subplots(1, 3, figsize = (18,6))
ap.plots.target_image(fig1, ax1[0], target_r)
ax1[0].set_title("r-band image")
ap.plots.target_image(fig1, ax1[1], target_W1)
ax1[1].set_title("W1-band image")
ap.plots.target_image(fig1, ax1[2], target_NUV)
ax1[2].set_title("NUV-band image")
plt.show()

In [ ]:
# The multiband model will need a target to try and fit, but now that we have multiple images the "target" is
# a Target_Image_List object which points to all three.
target_full = ap.image.Target_Image_List((target_r, target_W1, target_NUV))
# It doesn't really need any other information since everything is already available in the individual targets

In [ ]:
# To make things easy to start, lets just fit a sersic model to all three. In principle one can use arbitrary 
# group models designed for each band individually, but that would be unecessarily complex for a tutorial

model_r = ap.models.AutoProf_Model(
    name = "rband model",
    model_type = "sersic galaxy model",
    target = target_r,
)
model_W1 = ap.models.AutoProf_Model(
    name = "W1band model",
    model_type = "sersic galaxy model",
    target = target_W1,
)
model_NUV = ap.models.AutoProf_Model(
    name = "NUVband model",
    model_type = "sersic galaxy model",
    target = target_NUV,
)

# At this point we would just be fitting three separate models at the same time, not very interesting. Next 
# we add constraints so that some parameters are shared between all the models. It makes sense to fix 
# structure parameters while letting brightness parameters vary between bands so that's what we do here.
model_W1.add_equality_constraint(model_r, ["center", "q", "PA", "n", "Re"])
model_NUV.add_equality_constraint(model_r, ["center", "q", "PA", "n", "Re"])
# Now every model will have a unique Ie, but every other parameter is shared for all three

In [ ]:
# We can now make the multiband model object

model_full = ap.models.Multiband_Model(
    name = "LEDA 41136",
    model_list = [model_r, model_W1, model_NUV],
    target = target_full,
)

model_full.initialize()

In [ ]:
result = ap.fit.LM(model_full, method = 0, verbose = 1, relative_tolerance = 1e-5).fit() # here we set a very tight tolerance since the r-band image with 100X more pixels will mostly control the Chi^2, its not necessary but this is good insurance
print(result.message)

In [ ]:
# here we plot the results of the fitting, notice that each band has a different PSF and pixelscale. Also, notice
# that the colour bars represent significantly different ranges since each model was allowed to fit its own Ie.
# meanwhile the center, PA, q, and Re is the same for every model.
fig1, ax1 = plt.subplots(1, 3, figsize = (18,6))
ap.plots.model_image(fig1, ax1[0], model_r)
ax1[0].set_title("r-band model image")
ap.plots.model_image(fig1, ax1[1], model_W1)
ax1[1].set_title("W1-band model image")
ap.plots.model_image(fig1, ax1[2], model_NUV)
ax1[2].set_title("NUV-band model image")
plt.show()

In [ ]:
# We can also plot the residual images. As can be seen, the galaxy is fit in all three bands simultaneously
# with the majority of the light removed in all bands. A slight residual can be seen in the W1 and NUV bands.
# This is likely due to the very simplistic gaussian PSF used for the sake of this tutorial.
fig1, ax1 = plt.subplots(1, 3, figsize = (18,6))
ap.plots.residual_image(fig1, ax1[0], model_r)
ax1[0].set_title("r-band residual image")
ap.plots.residual_image(fig1, ax1[1], model_W1)
ax1[1].set_title("W1-band residual image")
ap.plots.residual_image(fig1, ax1[2], model_NUV)
ax1[2].set_title("NUV-band residual image")
plt.show()